# Clean Dataset for Model

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Clean data for modeling

pitch = pd.read_csv('../data/mlb-pitches.csv', index_col = [0])
pitch = pitch[['player_name', 'p_throws', 'pitch_type','release_speed', 'release_spin_rate', 'spin_axis', 
               'pfx_-x', 'pfx_z', 'bauer_units', 'effective_speed', 'release_pos_-x', 'release_pos_x', 
               'release_pos_z', 'release_extension', 'release_pos_y', 'plate_-x', 'plate_x', 'plate_z', 
               'type', 'balls','strikes', 'pitch_count', 'delta_run_exp', 'stand', 'bb_type', 'description', 
               'events', 'hit_distance_sc', 'launch_speed', 'launch_angle', 'launch_speed_angle', 'woba_value', 
               'woba_denom', 'xba', 'xwoba', 'babip_value', 'iso_value', 'at_bat_number', 'pitch_number', 
               'inning', 'inning_topbot', 'home_score', 'away_score', 'post_home_score', 'post_away_score', 
               'on_1b', 'on_2b', 'on_3b', 'outs_when_up']].copy()

pitch.dropna(subset = ['pitch_type', 'release_speed', 'release_spin_rate', 'pfx_-x', 'pfx_z',
                       'release_extension', 'release_pos_-x', 'release_pos_z', 'delta_run_exp'], inplace = True)

#Rename some columns
col_dict = {
    'release_speed': 'velo',
    'release_spin_rate': 'spin_rate',
    'launch_speed': 'exit_velo',
}

pitch.rename(columns = col_dict, inplace = True)
pitch.to_csv('../data/model-data.csv')

In [3]:
# Run Expectany Table

# 2010-2015 Run Expectancy
matrix = [[0, 0, 0, 0, 0.481], [1, 0, 0, 0, 0.859], [0, 1, 0, 0, 1.100], [1, 1, 0, 0, 1.437], 
          [0, 0, 1, 0, 1.350], [1, 0, 1, 0, 1.784], [0, 1, 1, 0, 1.964], [1, 1, 1, 0, 2.292], 
          [0, 0, 0, 1, 0.254], [1, 0, 0, 1, 0.509], [0, 1, 0, 1, 0.664], [1, 1, 0, 1, 0.884], 
          [0, 0, 1, 1, 0.950], [1, 0, 1, 1, 1.130], [0, 1, 1, 1, 1.376], [1, 1, 1, 1, 1.541],
          [0, 0, 0, 2, 0.098], [1, 0, 0, 2, 0.224], [0, 1, 0, 2, 0.319], [1, 1, 0, 2, 0.429], 
          [0, 0, 1, 2, 0.353], [1, 0, 1, 2, 0.478], [0, 1, 1, 2, 0.580], [1, 1, 1, 2, 0.752]]

re = pd.DataFrame(matrix, columns = ['on_1b', 'on_2b', 'on_3b', 'outs_when_up', 're'])
# re.head()
re.to_csv('../data/run_expectancy_table.csv')

count_re = pd.read_csv('../data/count-re.csv')
print(count_re.shape)
count_re.head()

(24, 5)


In [4]:
# Add RV to model-data.csv

data = pd.read_csv('../data/model-data.csv', index_col = [0])

# Add Arm angle
arm_angle = data.groupby(['player_name', 'pitch_type'], as_index=False)['release_pos_x', 'release_pos_z'].mean()
arm_angle = pd.DataFrame(arm_angle)
adj = (arm_angle['release_pos_z'] - arm_angle['release_pos_x']) * 0.7
opp = abs(arm_angle['release_pos_x'])
hyp = np.sqrt((opp ** 2) + (adj ** 2))
arm_angle['arm_angle'] = round(np.arccos(((adj ** 2 + hyp ** 2) - opp ** 2) / (2 * (adj * hyp))), 3) * 100
arm_angle['slot'] = ['overhead' if (x >= 0) & (x <= 40) else '3/4' if (x >= 41) & (x <= 60) else 'sidearm'
                     for x in arm_angle['arm_angle']]
arm_angle.drop(columns = ['release_pos_x', 'release_pos_z'], inplace = True)
data = pd.merge(data, arm_angle, how = 'left', on = ['player_name', 'pitch_type'])

data = data[data['events'] != 'wild_pitch']
data = data[data['events'] != 'passed_ball']
data = data[data['events'] != 'stolen_base_2b']

data['events'].replace(['sac_bunt', 'double_play', 'caught_stealing_2b', 'strikeout_double_play',
                        'other_out', 'sac_fly_double_play', 'pickoff_2b', 'pickoff_3b', 'triple_play'
                        'caught_stealing_home', 'pickoff_caught_stealing_2b', 'pickoff_3b', 
                        'sac_bunt_double_play', 'pickoff_caught_stealing_3b', 'pickoff_1b', 
                        'caught_stealing_3b', 'triple_play', 'caught_stealing_home', 
                        'strikeout'], 'field_out', inplace = True)
data['events'].replace(['catcher_interf'], 'field_error', inplace = True)

data['description'].replace(['called_strike', 'swinging_strike', 'swinging_strike_blocked', 'missed_bunt',
                             'foul'], 'strike', inplace = True)
data['description'].replace(['passed_ball', 'wild_pitch'], 'ball', inplace = True)

data['is_strike'] = [1 if x != 'B' else 0 for x in data['type']]
data['is_ball'] = [1 if x == 'B' else 0 for x in data['type']]

data['inning_topbot'] = data.inning_topbot.map({'Top': 0, 'Bot': 1})
data['on_1b'] = [1 if x > 1 else 0 for x in data['on_1b']]
data['on_2b'] = [1 if x > 1 else 0 for x in data['on_2b']]
data['on_3b'] = [1 if x > 1 else 0 for x in data['on_3b']]

data['final_pitch_ab'] = [1 if x == x else 0 for x in data['events']]
data['out_to_end_inning'] = [1 if x == 'field_out' and y == 2 else 0 for (x, y) 
                             in zip(data['events'], data['outs_when_up'])]
data['home_runs'] = data['post_home_score'] - data['home_score']
data['away_runs'] = data['post_away_score'] - data['away_score']
data['runs'] = data['home_runs'] + data['away_runs']

# Merge RE Table with data
data = pd.merge(data, re, how = 'left', on = ['on_1b', 'on_2b', 'on_3b', 'outs_when_up'])
data['re_change'] = round(-data['re'].diff(1), 3)
data['re_change'].fillna(-0.098, inplace = True)
data['re_end_state'] = data['re'] + data['re_change']
data['re_end_state'] = [0 if x == 1 else y for (x, y) in zip(data['out_to_end_inning'], data['re_end_state'])]
data['re_change'] = [-y if x == 1 else z for (x, y, z) in zip(data['out_to_end_inning'], 
                                                              data['re'], data['re_change'])]
data['re_change'].replace([-0.000], 0.000, inplace = True)
data['re24'] = data['re_change'] + data['runs']

# Create Linear Weights with average RE by events
mlb_lw = data.groupby(['events'], as_index = False)['re24'].mean()
lw_ball_in_play = pd.DataFrame(mlb_lw)
lw_ball_in_play['re24'] = round(lw_ball_in_play['re24'], 3)
lw_ball_in_play.rename(columns = {'re24': 'lin_weight_above_avg'}, inplace = True)

# For lin weights based on base out state 
data = pd.merge(data, lw_ball_in_play, how = 'left', on = ['events'])
data['lin_weight_above_outs'] = data['lin_weight_above_avg'] + 0.25
data['woba_scale'] = 1.209
data['lin_weights_above_avg_scale'] = round(data['lin_weight_above_avg'] * data['woba_scale'], 3)
data['lin_weights_above_outs_scale'] = round(data['lin_weight_above_outs'] * data['woba_scale'], 3)

# Merge ball strike count RE with data
data = pd.merge(data, count_re, how = 'left', on = ['pitch_count', 'is_strike', 'is_ball'])
data['count_re'] = data['wraa_change'] + data['re24']

# # Create Linear Weights with average RE by ball strike count
mlb_lw_bs = data.groupby(['pitch_count'], as_index = False)['count_re'].mean()
lw_bs = pd.DataFrame(mlb_lw_bs)
lw_bs['count_re'] = round(lw_bs['count_re'], 3)
lw_bs.rename(columns = {'count_re': 'bs_lin_weight'}, inplace = True)
data = pd.merge(data, lw_bs, how = 'left', on = ['pitch_count'])
data['bs_lin_weight_scale'] = round(data['bs_lin_weight'] * data['woba_scale'], 3)

data['lin_weights_above_avg_scale'].fillna(0, inplace = True)
data['lin_weights_above_outs_scale'].fillna(0, inplace = True)

#data['lin_weights_above_avg_scale'].fillna(data['bs_lin_weight_scale'], inplace = True)
#data['lin_weights_above_outs_scale'].fillna(data['bs_lin_weight_scale'], inplace = True)
#data['lin_weights_above_outs_scale'] = [y if x == 0.000 else x for (x, y) in 
#                                        zip(data['lin_weights_above_outs_scale'], data['bs_lin_weight_scale'])]

#data['lin_weights_above_avg_scale'].fillna(data['bs_lin_weight'], inplace = True)
#data['lin_weights_above_outs_scale'].fillna(data['bs_lin_weight'], inplace = True)
#data['lin_weights_above_outs_scale'] = [y if x == 0.000 else x for (x, y) in 
#                                        zip(data['lin_weights_above_outs_scale'], data['bs_lin_weight'])]

data['rv_above_avg'] = data['lin_weights_above_avg_scale'] + data['bs_lin_weight_scale'] # * 100
data['rv'] = data['lin_weights_above_outs_scale'] + data['bs_lin_weight_scale'] # * 100)

data.to_csv('../data/model-pitches-rv.csv')

pd.set_option('max_columns', None)
print(data.shape)
data.head()

(705396, 74)


,player_name,p_throws,pitch_type,velo,spin_rate,spin_axis,pfx_-x,pfx_z,bauer_units,effective_speed,release_pos_-x,release_pos_x,release_pos_z,release_extension,release_pos_y,plate_-x,plate_x,plate_z,type,balls,strikes,pitch_count,delta_run_exp,stand,bb_type,description,events,hit_distance_sc,exit_velo,launch_angle,launch_speed_angle,woba_value,woba_denom,xba,xwoba,babip_value,iso_value,at_bat_number,pitch_number,inning,inning_topbot,home_score,away_score,post_home_score,post_away_score,on_1b,on_2b,on_3b,outs_when_up,arm_angle,slot,is_strike,is_ball,final_pitch_ab,out_to_end_inning,home_runs,away_runs,runs,re,re_change,re_end_state,re24,lin_weight_above_avg,lin_weight_above_outs,woba_scale,lin_weights_above_avg_scale,lin_weights_above_outs_scale,woba,wraa_change,count_re,bs_lin_weight,bs_lin_weight_scale,rv_above_avg,rv
0,"Smith, Will",L,FF,92.3,2330.0,148.0,-8.28,16.56,25.243770,92.8,-1.40,1.40,6.80,6.5,54.03,0.69,-0.69,2.83,X,1,2,1-2,-0.073,R,ground_ball,hit_into_play,field_out,13.0,95.2,-13.0,2.0,0.0,1.0,0.174,0.158,0.0,0.0,61,4,9,0,5,0,5,0,0,0,0,2,36.0,overhead,1,0,1,1,0,0,0,0.098,-0.098,0.000,-0.098,-0.25,0.0,1.209,-0.302,0.0,0.223,-0.184,-0.282,-0.141,-0.170,-0.472,-0.170
1,"Smith, Will",L,SL,80.6,2254.0,315.0,9.24,5.76,27.965261,81.2,-1.60,1.60,6.64,6.4,54.15,0.71,-0.71,2.62,S,1,1,1-1,-0.027,R,NaN,strike,NaN,108.0,75.3,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,3,9,0,5,0,5,0,0,0,0,2,45.8,3/4,1,0,0,0,0,0,0,0.098,0.000,0.098,0.000,NaN,NaN,1.209,0.000,0.0,0.293,-0.058,-0.058,-0.006,-0.007,-0.007,-0.007
2,"Smith, Will",L,CU,75.5,1940.0,328.0,7.80,-6.12,25.695364,75.2,-1.46,1.46,6.88,6.2,54.34,0.04,-0.04,2.46,S,1,0,1-0,-0.020,R,NaN,strike,NaN,157.0,83.5,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,2,9,0,5,0,5,0,0,0,0,2,38.4,overhead,1,0,0,0,0,0,0,0.098,0.000,0.098,0.000,NaN,NaN,1.209,0.000,0.0,0.355,-0.051,-0.051,0.001,0.001,0.001,0.001
3,"Smith, Will",L,CU,75.0,2017.0,330.0,8.28,-8.28,26.893333,74.5,-1.53,1.53,6.83,5.9,54.61,-2.10,2.10,3.89,B,0,0,0-0,0.016,R,NaN,ball,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,1,9,0,5,0,5,0,0,0,0,2,38.4,overhead,0,1,0,0,0,0,0,0.098,0.000,0.098,0.000,NaN,NaN,1.209,0.000,0.0,0.314,0.034,0.034,-0.003,-0.004,-0.004,-0.004
4,"Smith, Will",L,FF,91.2,2281.0,143.0,-7.56,15.36,25.010965,90.9,-1.49,1.49,6.66,6.3,54.15,0.31,-0.31,2.80,X,1,0,1-0,-0.189,L,ground_ball,hit_into_play,field_out,9.0,93.3,-18.0,2.0,0.0,1.0,0.100,0.090,0.0,0.0,60,2,9,0,5,0,5,0,0,0,0,1,36.0,overhead,1,0,1,0,0,0,0,0.254,-0.156,0.098,-0.156,-0.25,0.0,1.209,-0.302,0.0,0.355,-0.051,-0.207,0.001,0.001,-0.301,0.001


In [5]:
data2 = data.copy()

data2['n_pitches'] = [1 if x is not np.nan else 0 for x in data2['pitch_type']]

print(data2.shape)
data2.head()

(705396, 75)


,player_name,p_throws,pitch_type,velo,spin_rate,spin_axis,pfx_-x,pfx_z,bauer_units,effective_speed,release_pos_-x,release_pos_x,release_pos_z,release_extension,release_pos_y,plate_-x,plate_x,plate_z,type,balls,strikes,pitch_count,delta_run_exp,stand,bb_type,description,events,hit_distance_sc,exit_velo,launch_angle,launch_speed_angle,woba_value,woba_denom,xba,xwoba,babip_value,iso_value,at_bat_number,pitch_number,inning,inning_topbot,home_score,away_score,post_home_score,post_away_score,on_1b,on_2b,on_3b,outs_when_up,arm_angle,slot,is_strike,is_ball,final_pitch_ab,out_to_end_inning,home_runs,away_runs,runs,re,re_change,re_end_state,re24,lin_weight_above_avg,lin_weight_above_outs,woba_scale,lin_weights_above_avg_scale,lin_weights_above_outs_scale,woba,wraa_change,count_re,bs_lin_weight,bs_lin_weight_scale,rv_above_avg,rv,n_pitches
0,"Smith, Will",L,FF,92.3,2330.0,148.0,-8.28,16.56,25.243770,92.8,-1.40,1.40,6.80,6.5,54.03,0.69,-0.69,2.83,X,1,2,1-2,-0.073,R,ground_ball,hit_into_play,field_out,13.0,95.2,-13.0,2.0,0.0,1.0,0.174,0.158,0.0,0.0,61,4,9,0,5,0,5,0,0,0,0,2,36.0,overhead,1,0,1,1,0,0,0,0.098,-0.098,0.000,-0.098,-0.25,0.0,1.209,-0.302,0.0,0.223,-0.184,-0.282,-0.141,-0.170,-0.472,-0.170,1
1,"Smith, Will",L,SL,80.6,2254.0,315.0,9.24,5.76,27.965261,81.2,-1.60,1.60,6.64,6.4,54.15,0.71,-0.71,2.62,S,1,1,1-1,-0.027,R,NaN,strike,NaN,108.0,75.3,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,3,9,0,5,0,5,0,0,0,0,2,45.8,3/4,1,0,0,0,0,0,0,0.098,0.000,0.098,0.000,NaN,NaN,1.209,0.000,0.0,0.293,-0.058,-0.058,-0.006,-0.007,-0.007,-0.007,1
2,"Smith, Will",L,CU,75.5,1940.0,328.0,7.80,-6.12,25.695364,75.2,-1.46,1.46,6.88,6.2,54.34,0.04,-0.04,2.46,S,1,0,1-0,-0.020,R,NaN,strike,NaN,157.0,83.5,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,2,9,0,5,0,5,0,0,0,0,2,38.4,overhead,1,0,0,0,0,0,0,0.098,0.000,0.098,0.000,NaN,NaN,1.209,0.000,0.0,0.355,-0.051,-0.051,0.001,0.001,0.001,0.001,1
3,"Smith, Will",L,CU,75.0,2017.0,330.0,8.28,-8.28,26.893333,74.5,-1.53,1.53,6.83,5.9,54.61,-2.10,2.10,3.89,B,0,0,0-0,0.016,R,NaN,ball,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,1,9,0,5,0,5,0,0,0,0,2,38.4,overhead,0,1,0,0,0,0,0,0.098,0.000,0.098,0.000,NaN,NaN,1.209,0.000,0.0,0.314,0.034,0.034,-0.003,-0.004,-0.004,-0.004,1
4,"Smith, Will",L,FF,91.2,2281.0,143.0,-7.56,15.36,25.010965,90.9,-1.49,1.49,6.66,6.3,54.15,0.31,-0.31,2.80,X,1,0,1-0,-0.189,L,ground_ball,hit_into_play,field_out,9.0,93.3,-18.0,2.0,0.0,1.0,0.100,0.090,0.0,0.0,60,2,9,0,5,0,5,0,0,0,0,1,36.0,overhead,1,0,1,0,0,0,0,0.254,-0.156,0.098,-0.156,-0.25,0.0,1.209,-0.302,0.0,0.355,-0.051,-0.207,0.001,0.001,-0.301,0.001,1


In [6]:
avg_metrics = data2.groupby(['player_name', 'p_throws', 'pitch_type'], 
                            as_index = False)['velo', 'spin_rate', 'pfx_-x', 'pfx_z', 
                                              'release_pos_-x','release_pos_z',
                                              'release_extension', 
                                              'arm_angle'].mean()
avg_metrics = pd.DataFrame(avg_metrics)

sum_rv = data2.groupby(['player_name', 'p_throws', 'pitch_type'], 
                       as_index = False)['n_pitches', 'rv'].sum()
rv = pd.merge(avg_metrics, sum_rv, how = 'left', on = ['player_name', 'p_throws', 'pitch_type'])
rv['RV/100'] = round(((rv['rv'] / rv['n_pitches']) * 100), 2)
rv['velo'] = round(rv['velo'], 1)
rv['spin_rate'] = round(rv['spin_rate'])
rv['pfx_-x'] = round(rv['pfx_-x'], 2)
rv['pfx_z'] = round(rv['pfx_z'], 2)
rv['release_extension'] = round(rv['release_extension'], 2)
rv['release_pos_-x'] = round(rv['release_pos_-x'], 2)
rv['release_pos_z'] = round(rv['release_pos_z'], 2)
print(rv.shape)
rv.head()

rv.to_csv('../data/test-rv.csv')

(3469, 14)


In [22]:
avg_metrics2 = data2.groupby(['player_name', 'p_throws', 'pitch_type'], 
                            as_index = False)['velo', 'spin_rate', 'pfx_-x', 'pfx_z', 
                                              'release_pos_-x','release_pos_z',
                                              'release_extension', 
                                              'arm_angle', 'rv'].mean()
avg_metrics2 = pd.DataFrame(avg_metrics2)

sum_pitches2 = data2.groupby(['player_name', 'p_throws', 'pitch_type'], 
                       as_index = False)['rv', 'n_pitches'].sum()
rv2 = pd.merge(avg_metrics2, sum_pitches2, how = 'left', on = ['player_name', 'p_throws', 'pitch_type'])
# rv2['RV/100'] = round(((rv['rv'] / rv['n_pitches']) * 100), 2)
rv2['velo'] = round(rv2['velo'], 1)
rv2['spin_rate'] = round(rv2['spin_rate'])
rv2['pfx_-x'] = round(rv2['pfx_-x'], 2)
rv2['pfx_z'] = round(rv2['pfx_z'], 2)
rv2['release_extension'] = round(rv2['release_extension'], 2)
rv2['release_pos_-x'] = round(rv2['release_pos_-x'], 2)
rv2['release_pos_z'] = round(rv2['release_pos_z'], 2)
rv2.rename(columns = {'rv_x': 'avg_rv', 'rv_y': 'rv'}, inplace = True)
rv2.to_csv('../data/test-rv2.csv')

In [29]:
rv2['pp'] = rv2['rv'] / rv2['n_pitches']
rv2['pp'] = rv2['avg_rv'] * rv2['n_pitches']
print(rv2.shape)
rv2.head()

(3469, 15)


,player_name,p_throws,pitch_type,velo,spin_rate,pfx_-x,pfx_z,release_pos_-x,release_pos_z,release_extension,arm_angle,avg_rv,rv,n_pitches,pp
0,"Abad, Fernando",L,CH,75.1,1638.0,-10.65,14.23,-0.71,6.42,5.88,17.7,-0.022870,-1.052,46,-1.052
1,"Abad, Fernando",L,FC,82.6,2310.0,3.38,2.74,-1.14,6.02,5.86,32.2,0.048750,0.780,16,0.780
2,"Abad, Fernando",L,FF,91.7,2112.0,-5.34,16.20,-0.45,6.18,5.86,11.3,0.157611,2.837,18,2.837
3,"Abad, Fernando",L,KC,76.4,2502.0,7.23,-9.18,-0.73,6.26,5.63,18.8,0.043000,3.354,78,3.354
4,"Abad, Fernando",L,SI,92.4,2145.0,-13.19,12.92,-0.56,6.29,5.83,13.9,0.054730,7.717,141,7.717


In [14]:
sum_pitches3 = data2.groupby(['player_name', 'p_throws', 'pitch_type'], 
                       as_index = False)['rv'].sum()
sum_pitches3.head()

,player_name,p_throws,pitch_type,rv
0,"Abad, Fernando",L,CH,-1.052
1,"Abad, Fernando",L,FC,0.780
2,"Abad, Fernando",L,FF,2.837
3,"Abad, Fernando",L,KC,3.354
4,"Abad, Fernando",L,SI,7.717


In [24]:
avg_metrics2.head()

,player_name,p_throws,pitch_type,velo,spin_rate,pfx_-x,pfx_z,release_pos_-x,release_pos_z,release_extension,arm_angle,rv
0,"Abad, Fernando",L,CH,75.134783,1637.739130,-10.651304,14.225217,-0.713696,6.423696,5.880435,17.7,-0.022870
1,"Abad, Fernando",L,FC,82.593750,2310.375000,3.375000,2.745000,-1.140000,6.022500,5.856250,32.2,0.048750
2,"Abad, Fernando",L,FF,91.688889,2112.111111,-5.340000,16.200000,-0.453333,6.175556,5.855556,11.3,0.157611
3,"Abad, Fernando",L,KC,76.408974,2501.858974,7.230769,-9.180000,-0.734872,6.259744,5.625641,18.8,0.043000
4,"Abad, Fernando",L,SI,92.356028,2144.503546,-13.185532,12.923404,-0.561064,6.294468,5.832624,13.9,0.054730


In [27]:
data.loc[(data['player_name'] == 'Abad, Fernando') & (data['pitch_type'] == 'FF')].describe()

,velo,spin_rate,spin_axis,pfx_-x,pfx_z,bauer_units,effective_speed,release_pos_-x,release_pos_x,release_pos_z,release_extension,release_pos_y,plate_-x,plate_x,plate_z,balls,strikes,delta_run_exp,hit_distance_sc,exit_velo,launch_angle,launch_speed_angle,woba_value,woba_denom,xba,xwoba,babip_value,iso_value,at_bat_number,pitch_number,inning,inning_topbot,home_score,away_score,post_home_score,post_away_score,on_1b,on_2b,on_3b,outs_when_up,arm_angle,is_strike,is_ball,final_pitch_ab,out_to_end_inning,home_runs,away_runs,runs,re,re_change,re_end_state,re24,lin_weight_above_avg,lin_weight_above_outs,woba_scale,lin_weights_above_avg_scale,lin_weights_above_outs_scale,woba,wraa_change,count_re,bs_lin_weight,bs_lin_weight_scale,rv_above_avg,rv
count,18.000000,18.000000,18.000000,18.000000,18.00000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.0000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,8.000000,8.00000,8.000000,4.000000,6.000000,6.0,4.000000,4.000000,6.000000,6.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.0,18.000000,18.000000,1.800000e+01,18.000000,18.000000,18.000000,18.0,18.000000,18.0,18.000000,18.000000,18.000000,18.000000,18.000000,6.00000,6.00000,1.800000e+01,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000,18.000000
mean,91.688889,2112.111111,154.000000,-5.340000,16.20000,22.985762,91.127778,-0.453333,0.453333,6.175556,5.855556,54.6400,-0.327778,0.327778,2.856667,1.666667,1.777778,0.175778,187.750000,85.71250,16.000000,4.500000,0.958333,1.0,0.607500,0.843250,0.500000,0.666667,58.722222,4.666667,6.611111,0.666667,5.888889,3.611111,5.944444,3.611111,0.444444,0.0,0.055556,1.166667,1.130000e+01,0.500000,0.500000,0.333333,0.0,0.055556,0.0,0.055556,0.363389,0.112278,0.475667,0.167833,0.52350,0.77350,1.209000e+00,0.211056,0.311722,0.282611,-0.076667,0.091167,-0.127667,-0.154111,0.056944,0.157611
std,8.190015,248.948350,22.210226,0.999129,1.50275,1.271650,8.120548,0.218982,0.218982,0.153899,0.182216,0.1757,1.023314,1.023314,0.839180,0.840168,0.548319,0.314820,151.667635,17.49926,29.061511,1.914854,0.657584,0.0,0.349069,0.573727,0.547723,1.211060,11.166155,1.455214,1.144752,0.485071,2.987993,2.913233,3.038425,2.913233,0.511310,0.0,0.235702,0.785905,3.655712e-15,0.514496,0.514496,0.485071,0.0,0.235702,0.0,0.235702,0.238144,0.270524,0.443115,0.339897,0.54205,0.54205,2.284820e-16,0.469736,0.576277,0.058723,0.167219,0.309664,0.058597,0.070748,0.461266,0.562984
min,59.200000,1226.000000,139.000000,-7.080000,14.16000,20.709459,58.900000,-0.880000,0.030000,5.910000,5.600000,54.1400,-1.830000,-1.410000,0.810000,0.000000,0.000000,-0.142000,3.000000,67.60000,-31.000000,2.000000,0.000000,1.0,0.089000,0.089000,0.000000,0.000000,26.000000,1.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.130000e+01,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.098000,-0.156000,0.098000,-0.156000,-0.25000,0.00000,1.209000e+00,-0.302000,0.000000,0.223000,-0.318000,-0.382000,-0.168000,-0.203000,-0.505000,-0.203000
25%,93.025000,2059.750000,144.750000,-6.090000,14.97000,22.173908,92.425000,-0.537500,0.295000,6.055000,5.800000,54.5825,-1.257500,-0.350000,2.412500,1.000000,2.000000,0.004500,83.250000,71.32500,4.000000,3.500000,0.750000,1.0,0.579500,0.550250,0.000000,0.000000,52.000000,4.000000,6.250000,0.000000,4.000000,3.000000,4.000000,3.000000,0.000000,0.0,0.000000,1.000000,1.130000e+01,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.224000,0.000000,0.129500,0.000000,0.35000,0.60000,1.209000e+00,0.000000,0.000000,0.223000,-0.226000,-0.040250,-0.168000,-0.203000,-0.173750,-0.173750
50%,93.800000,2112.000000,150.000000,-5.640000,16.08000,22.654584,93.150000,-0.490000,0.490000,6.180000,5.800000,54.6800,-0.445000,0.445000,2.865000,2.000000,2.000000,0.043000,181.500000,79.25000,15.000000,5.000000,0.900000,1.0,0.746000,0.995000,0.500000,0.000000,64.500000,4.500000,7.000000,1.000000,6.500000,3.

In [26]:
avg_metrics2.loc[(avg_metrics2['player_name'] == 'Abad, Fernando') & (avg_metrics2['pitch_type'] == 'FF')].describe()

,velo,spin_rate,pfx_-x,pfx_z,release_pos_-x,release_pos_z,release_extension,arm_angle,rv
count,1.000000,1.000000,1.00,1.0,1.000000,1.000000,1.000000,1.0,1.000000
mean,91.688889,2112.111111,-5.34,16.2,-0.453333,6.175556,5.855556,11.3,0.157611
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,91.688889,2112.111111,-5.34,16.2,-0.453333,6.175556,5.855556,11.3,0.157611
25%,91.688889,2112.111111,-5.34,16.2,-0.453333,6.175556,5.855556,11.3,0.157611
50%,91.688889,2112.111111,-5.34,16.2,-0.453333,6.175556,5.855556,11.3,0.157611
75%,91.688889,2112.111111,-5.34,16.2,-0.453333,6.175556,5.855556,11.3,0.157611
max,91.688889,2112.111111,-5.34,16.2,-0.453333,6.175556,5.855556,11.3,0.157611
